In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import random

from transformers import PretrainedConfig
from model.visual_token_embedding import VisualTokenEmbedding
from utils.visualization import visualize_masks
from utils.logging import get_params_count_summary
from tokenizer.directsam import DirectSAMTokenizer

image_dir = '/home/dchenbs/workspace/datasets/coco2017/images/val2017'

# image_dir = '/home/dchenbs/workspace/datasets/Adobe_EntitySeg/images/entity_01_11580'
# image_dir = '/home/dchenbs/workspace/datasets/Adobe_EntitySeg/images/entity_02_11598'
# image_dir = '/home/dchenbs/workspace/datasets/Adobe_EntitySeg/images/entity_03_10049'
# image_dir = '/home/dchenbs/workspace/datasets/CLEVR_v1.0/images/test'
# image_dir =  '/home/dchenbs/workspace/datasets/sa1b/sa_000001'
# image_dir =  '/home/dchenbs/workspace/subobjects-dev/assets/examples'

# walk through the image_dir and get all the image paths
image_paths = []
for root, dirs, files in os.walk(image_dir):
    for file in files:
        if file.endswith('.jpg') or file.endswith('.png'):
            image_paths.append(os.path.join(root, file))

print(len(image_paths))

In [ ]:

image_resolution = 1024

visual_tokenizer = DirectSAMTokenizer(
    ckpt="chendelong/DirectSAM-tiny-distilled-30ep-plus-50ep-1024px-0910",
    threshold=0.1,
    image_resolution=image_resolution,
    max_tokens=128,
    device="cuda"
)

In [ ]:
visual_token_embedding_config = PretrainedConfig.from_dict({
    "image_resolution": image_resolution,
    "token_resolution": 16,
    'embedding_dim': 384,

    # "vision_encoder_type": "rgb_pixel",
    # "vision_encoder_name": "rgb_pixel",
    
    "vision_encoder_type": "diffusers_vae",
    "vision_encoder_name": "chendelong/stable-diffusion-3-medium-vae",

    # https://huggingface.co/models?sort=trending&search=facebook%2Fdinov2
    # "vision_encoder_type": "hf_autobacbone",
    # "vision_encoder_name": "facebook/dinov2-giant", # small, base, large, giant
    
    # # # https://huggingface.co/models?search=facebook/convnextv2
    # "vision_encoder_type": "hf_autobacbone",
    # "vision_encoder_name": "facebook/convnextv2-tiny-22k-384/stage2", 

    # # https://huggingface.co/models?search=microsoft/resnet
    # "vision_encoder_type": "hf_autobacbone",
    # "vision_encoder_name": "microsoft/resnet-50", # 18, 34, 50, 101

    # https://huggingface.co/timm
    # "vision_encoder_type": "timm_backbone",
    # "vision_encoder_name": "convnext_large_mlp.clip_laion2b_soup_ft_in12k_in1k_320/-2", 
})


visual_token_embedding = VisualTokenEmbedding(visual_token_embedding_config).cuda()

print(visual_token_embedding.device, visual_token_embedding.dtype)

print(visual_token_embedding.vision_encoder.feature_channels, 'channels')

In [ ]:
# get an image
from PIL import Image
import requests

# url = "http://images.cocodataset.org/val2017/000000039769.jpg"
# image = Image.open(requests.get(url, stream=True).raw).convert("RGB")

image_path = random.sample(image_paths, 4)
images = [Image.open(path).convert("RGB") for path in image_path]

feature_maps = visual_token_embedding.vision_encoder(images)
print(feature_maps.shape, feature_maps.dtype, feature_maps.device)
feature_maps = feature_maps.cpu().numpy()

In [ ]:
from sklearn.decomposition import PCA
import numpy as np

def apply_pca(feature_maps):
    # feature_maps: N, h, w, feature_dim
    if feature_maps.shape[0] > 3:
        n_components = 3

        X = np.swapaxes(feature_maps, -1, 0)
        X = np.reshape(X, (-1, X.shape[-1]))

        pca = PCA(n_components=n_components)
        pca.fit(X)
        projection = pca.components_ # n_components, feature_dim

        # project the features to the PCA space -> n_images, n_components, h, w
        feature_maps = np.matmul(np.swapaxes(feature_maps, -1, 0), projection.T)
        feature_maps = np.swapaxes(feature_maps, -1, 0)

    # normalize the features to [0, 1]
    feature_maps = (feature_maps - np.min(feature_maps)) / (np.max(feature_maps) - np.min(feature_maps))
    return feature_maps


In [ ]:
import matplotlib.pyplot as plt

for b, image in enumerate(images):
    plt.figure(figsize=(20, 10))

    plt.subplot(1, 2, 1)
    plt.imshow(image.resize((512, 512)))
    plt.axis("off")

    plt.subplot(1, 2, 2)
    print(feature_maps.shape)
    feature_maps_rgb = np.transpose(apply_pca(feature_maps[b]), (1, 2, 0))
    plt.imshow(feature_maps_rgb)
    plt.axis("off")

    plt.show()


In [ ]:
batch_masks = visual_tokenizer(images)

for b, image in enumerate(images):
    image = image.resize((image_resolution, image_resolution))

    plt.figure(figsize=(20, 10))
    plt.subplot(1, 2, 1)
    plt.imshow(image)

    plt.subplot(1, 2, 2)
    plt.imshow(visualize_masks(image, batch_masks[b][:64]))
    plt.title(len(batch_masks[0]))
    plt.show()

In [ ]:
valid_masks = 0
for mask in batch_masks[0]:
    if mask.sum() > 0:
        valid_masks += 1
print(valid_masks, 'valid masks')

In [ ]:
roi_boxes, roi_masks, embeddings = visual_token_embedding(images, batch_masks)
print('embeddings', embeddings.shape)
print('roi_boxes', roi_boxes.shape)
print('roi_masks', roi_masks.shape)

roi_boxes = roi_boxes.cpu().numpy()
embeddings = embeddings.cpu().numpy()
roi_masks = roi_masks.cpu().numpy()

embeddings = (embeddings - np.min(embeddings)) / (np.max(embeddings) - np.min(embeddings))

In [ ]:
C = visual_token_embedding.vision_encoder.feature_channels
token_resolution = visual_token_embedding.config.token_resolution

for b, image in enumerate(images):
    image = image.resize((image_resolution, image_resolution))
    down_sample_ratio = image_resolution // np.max(roi_boxes[b] + 1)
    for i in range(4):
        plt.figure(figsize=(20, 8))
        plt.subplot(1, 6, 1)
        plt.imshow(image)

        plt.subplot(1, 6, 2)
        plt.imshow(batch_masks[b, i], cmap='inferno')

        x1, y1, x2, y2 = roi_boxes[b][i] * down_sample_ratio
        plt.plot([x1, x2, x2, x1, x1], [y1, y1, y2, y2, y1], 'r')
        plt.title(f'ROI')

        plt.subplot(1, 6, 3)
        plt.title('embedding')
        embedding = embeddings[b][i]
        embedding = embedding.reshape(C, token_resolution, token_resolution)

        feature_maps_rgb = np.transpose(apply_pca(embedding), (1, 2, 0))
        plt.imshow(feature_maps_rgb * roi_masks[b][i][:, :, None])

        plt.show()